# Instructor Do: Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/josearturomorasoto/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]



Total articles about Facebook Libra: 3149


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Romain Dillet',
 'title': 'What is the Libra Association going to do, really?',
 'description': 'When Facebook unveiled Libra a few days ago, the company also announced the Libra Association, a not-for-profit that will oversee all things Libra. Facebook wants to make sure that everyone is aware that Libra was created by Facebook but isn’t controlled by F…',
 'url': 'http://techcrunch.com/2019/06/26/what-is-the-libra-association-going-to-do-really/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2019/06/GettyImages-1146087387.jpg?w=590',
 'publishedAt': '2019-06-26T17:26:16Z',
 'content': 'When Facebook unveiledLibra a few days ago, the company also announced the Libra Association, a not-for-profit that will oversee all things Libra. Facebook wants to make sure that everyone is aware that Libra was created by Facebook but isnt controlled by Fac… [+2026 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,text,compound,positive,negative,neutral
0,2019-06-26,"When Facebook unveiledLibra a few days ago, th...",0.2846,0.073,0.0,0.927
1,2019-06-19,"On Tuesday Facebook announced Libra, a cryptoc...",0.7196,0.152,0.0,0.848
2,2019-07-03,The $10 million entry fee to join the Facebook...,0.2960,0.052,0.0,0.948
3,2019-07-03,"""We write to request that Facebook and its par...",0.3612,0.056,0.0,0.944
4,2019-06-18,Illustration by Alex Castro / The Verge\r\nAs ...,0.0000,0.000,0.0,1.000


In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()



,compound,positive,negative,neutral
count,99.000000,99.000000,99.000000,99.000000
mean,0.260642,0.067091,0.017646,0.915273
std,0.352937,0.062159,0.034538,0.073980
min,-0.735100,0.000000,0.000000,0.710000
25%,0.000000,0.000000,0.000000,0.871500
50%,0.226300,0.057000,0.000000,0.926000
75%,0.548500,0.113000,0.014500,1.000000
max,0.897900,0.225000,0.153000,1.000000
